<a href="https://colab.research.google.com/github/nacho1907/Bases/blob/main/Ejercicios_vivo_clase_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import scipy.stats as st
import pandas as pd
import matplotlib.pyplot as plt

# Introducción a la Inferencia Causal

# Clase 1 - Ejercicio en Vivo

> Gabo y Wen fueron a comer.
>
> Gabo comió mariscos. Wendy comió pasta.
>
> Gabo se sintió mal... ¿Fueron los mariscos?
>
> Wendy no se sintió mal.
>
> ¿Conclusión?

Inspirados en esta típica situación, consideremos un modelo sencillo.

Supongamos que conocemos el modelo generador de datos: el dolor de panza depende
1. Un poco de comer un plato en mal estado o no, y
2. considerablemente más de la salud previa.

Concretamente, postulemos lo siguiente:

- Tenemos una variable aleatoria $Y= \text{malestar}$ que es el "resultado".
- El "tratamiento" es comer mariscos, $T = \text{mariscos}$. Es una variable binaria, puede valer $0$ si el individuo no come mariscos o $1$ si come mariscos.
- La variable confusora es $X = \text{salud_previa}$. También la tomamos como variable binaria, que vale $1$ si el individuo tiene buena salud previa o $0$ si no.

Vamos a tomar a $\text{malestar}$ como variables numérica que puede tomar distintos valores, y mientras más alto es el valor, mayor es el malestar del individuo.


Supongamos que los mariscos estaban efectivamente en mal estado. Postulemos entonces la siguiente relación para el $i$-ésimo individuo:

$$\text{bienestar}_i = - 2 \text{mariscos}_i + 3 \text{salud_previa}_i + u_i.$$

Recordar que la variable aleatoria $u_i$ es el _término de ruido_.

**Atención: esta ecuación es lo que en la realidad nunca podemos conocer. Es el modelo generador de los datos** (junto con las dos ecuaciones de abajo que indican cómo se generan $\text{salud_previa}$ y $\text{mariscos}$).

Para terminar de especificar el modelo y poder generar los datos, tenemos que dar distribuciones de probabilidad sobre las variables.

Digamos que la probabilidad de que un individuo tenga buena salud previa es $p = 0.7$. Luego

$$\text{salud_previa} = \text{Bernoulli}(p = 0.7)$$

Supongamos además que todas las personas que se venían sintiendo bien se animan a probar mariscos, y todas las que se venían sintiendo mal deciden _no_ comer mariscos. Es decir que

$$\text{mariscos} = \text{salud_previa}.$$

Por último, vamos a tomar $u$ como una variable aleatoria normal con desviación estándar $1$.




## Ejercicio 1
Supongamos que queremos estudiar el efecto de comer mariscos en el malestar para 200 individuos. Simular los datos correspondientes a esta situación. Luego, con esos datos, calcular el bienestar promedio para los que comen mariscos versus para los que no comen mariscos. ¿Qué concluiría?


## Ejercicio 1
Supongamos que queremos estudiar el efecto de comer mariscos en el malestar para 200 individuos. Simular los datos correspondientes a esta situación. Luego, con esos datos, calcular el bienestar promedio para los que comen mariscos versus para los que no comen mariscos. ¿Qué concluiría?

### Solución

In [1]:
def correr_simulacion_1(p = 0.7, n = 200):
  salud_previa = st.bernoulli.rvs(p, size = n)
  mariscos = salud_previa
  bienestar = - 2 * mariscos + 3 * salud_previa +  np.random.normal(0, 1, size = n)
  return pd.DataFrame({"salud previa": salud_previa, "mariscos": mariscos, "bienestar": bienestar})

In [4]:
datos_ej1 = correr_simulacion_1()
bienestar_mariscos = datos_ej1[datos_ej1["mariscos"] == 1]["bienestar"].mean()
bienestar_no_mariscos = datos_ej1[datos_ej1["mariscos"] == 0]["bienestar"].mean()
print("Bienestar de los que comen mariscos:", bienestar_mariscos)
print("Bienestar de los que no comen mariscos:", bienestar_no_mariscos)

Bienestar de los que comen mariscos: 1.0249099419001457
Bienestar de los que no comen mariscos: 0.07220272024008963


In [6]:
datos_ej1.head(2)

,salud previa,mariscos,bienestar
0,1,1,2.063759
1,0,0,-0.229184


Vemos que los que comieron mariscos se sienten mejor que los que no comieron, incluso aunque sabemos que en realidad los mariscos sí tenían un impacto negativo en el bienestar.

## Ejercicio 2: Armando un experimento

Escriba una simulación de un **experimento**, donde de forma aleatoria a cada persona se le da un plato de mariscos con probabilidad $0.5$.

(Notar que las muestras que obtendremos no son muestras observacionales sino _experimentales_).

Cree un set de datos (muestras experimentales) a partir de esta simulación.

In [33]:
def exp_mariscos(n):
  come = st.bernoulli.rvs(0.5, size = n)
  return come

In [34]:
exp_mariscos(5)

array([0, 1, 1, 1, 1])

In [35]:
def correr_simulacion_2(p = 0.7, n = 200):
  salud_previa = st.bernoulli.rvs(p, size = n)
  mariscos = exp_mariscos(n)
  bienestar = - 2 * mariscos + 3 * salud_previa +  np.random.normal(0, 1, size = n)
  return pd.DataFrame({"salud previa": salud_previa, "mariscos": mariscos, "bienestar": bienestar})

In [37]:
datos_ej2 = correr_simulacion_2(0.7, 200)
bienestar_mariscos2 = datos_ej2[datos_ej2["mariscos"] == 1]["bienestar"].mean()
bienestar_no_mariscos2 = datos_ej2[datos_ej2["mariscos"] == 0]["bienestar"].mean()
print("Bienestar de los que comen mariscos:", bienestar_mariscos2)
print("Bienestar de los que no comen mariscos:", bienestar_no_mariscos2)

Bienestar de los que comen mariscos: -0.14534850494185678
Bienestar de los que no comen mariscos: 1.7821245900448814



## Ejercicio 3: Estimando el efecto causal a través de la media

Dadas la muestra de datos experimentales, calcule el efecto causal utilizando la diferencia de medias entre los tratados (grupo de tratamiento) y los no tratados (grupo de control). ¿Qué concluiría?


In [38]:
df_exp_trat = datos_ej2[datos_ej2['mariscos']==1]
df_exp_cont = datos_ej2[datos_ej2['mariscos']==0]

In [45]:
bienestar_fin = df_exp_trat.mean() - df_exp_cont.mean()

In [46]:
bienestar_fin['bienestar']

-1.9274730949867382

## Ejercicio 4: Visualizando resultados de inferencia causal.

Utilizando la biblioteca `matplotlib`, grafique la distribución de los efectos causales obtenidos a través de 100 simulaciones de experimentos aleatorios como los del ejercicio 2. Puede también dar algunos estadísticos sumarios como el promedio muestral y la desviación estándar.

In [47]:
import matplotlib.pyplot as plt